After a lot of research and letting ideas go, I settled on the Loan Prediction Problem

https://www.kaggle.com/altruistdelhite04/loan-prediction-problem-dataset

My first choice was to attempt to predict the next big video game trend on youtube for content creators, but I was unable to find a dataset that I could use. If I could find this data, I would prefer to attempt that challenge (I couldn't figure out how to use the data from the Google YouTube 8M, and it looks like it's 31 GBs)

https://research.google.com/youtube8m/

Purpose of Project
---

Using provided data, create a Predictive Model to determine loan approval or denial.

Step 1:

Import and clean data.

In [321]:
#Mount Drive

In [322]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [323]:
#Import cleaning and exploratory mods
#Read in data

In [324]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ltrain_df = pd.read_csv('/content/train.csv', index_col='Loan_ID')
ltest_df = pd.read_csv('/content/test.csv', index_col='Loan_ID')

print(ltrain_df.head())
print(ltest_df.head())

         Gender Married Dependents  ... Credit_History Property_Area  Loan_Status
Loan_ID                             ...                                          
LP001002   Male      No          0  ...            1.0         Urban            Y
LP001003   Male     Yes          1  ...            1.0         Rural            N
LP001005   Male     Yes          0  ...            1.0         Urban            Y
LP001006   Male     Yes          0  ...            1.0         Urban            Y
LP001008   Male      No          0  ...            1.0         Urban            Y

[5 rows x 12 columns]
         Gender Married  ... Credit_History Property_Area
Loan_ID                  ...                             
LP001015   Male     Yes  ...            1.0         Urban
LP001022   Male     Yes  ...            1.0         Urban
LP001031   Male     Yes  ...            1.0         Urban
LP001035   Male     Yes  ...            NaN         Urban
LP001051   Male      No  ...            1.0         Urb

In [325]:
#Check data for dtype and nulls in loantrain_df

In [326]:
ltrain_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, LP001002 to LP002990
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             601 non-null    object 
 1   Married            611 non-null    object 
 2   Dependents         599 non-null    object 
 3   Education          614 non-null    object 
 4   Self_Employed      582 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   600 non-null    float64
 9   Credit_History     564 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 62.4+ KB


In [327]:
ltrain_df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [328]:
ltrain_df.duplicated().any()

False

In [329]:
ltrain_df['Gender'].value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [330]:
#Replaced Nulls with majority value
ltrain_df['Gender'].fillna(value = 'Male', inplace = True)
ltrain_df['Gender'].value_counts()

Male      502
Female    112
Name: Gender, dtype: int64

In [331]:
ltrain_df['Married'].value_counts()

Yes    398
No     213
Name: Married, dtype: int64

In [332]:
#Replaced Nulls with No
ltrain_df['Married'].fillna(value = 'No', inplace = True)
ltrain_df['Married'].value_counts()

Yes    398
No     216
Name: Married, dtype: int64

In [333]:
ltrain_df['Dependents'].value_counts()

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64

In [334]:
#Replaced Nulls with 0
ltrain_df['Dependents'].fillna(value = 0, inplace = True)
ltrain_df['Dependents'].value_counts()

0     345
1     102
2     101
3+     51
0      15
Name: Dependents, dtype: int64

In [335]:
ltrain_df['Education'].value_counts()

Graduate        480
Not Graduate    134
Name: Education, dtype: int64

In [336]:
ltrain_df['Self_Employed'].value_counts()

No     500
Yes     82
Name: Self_Employed, dtype: int64

In [337]:
#Replaced Nulls with No
ltrain_df['Self_Employed'].fillna(value = 'No', inplace = True)
ltrain_df['Self_Employed'].value_counts()

No     532
Yes     82
Name: Self_Employed, dtype: int64

In [338]:
ltrain_df['Property_Area'].value_counts()

Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64

In [339]:
ltrain_df['Loan_Status'].value_counts()

Y    422
N    192
Name: Loan_Status, dtype: int64

In [340]:
ltrain_df['Loan_Amount_Term'].value_counts()

360.0    512
180.0     44
480.0     15
300.0     13
84.0       4
240.0      4
120.0      3
36.0       2
60.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [341]:
#Replaced Nulls with majority value
ltrain_df['Loan_Amount_Term'].fillna(value = 360, inplace = True)
ltrain_df['Loan_Amount_Term'].value_counts()

360.0    526
180.0     44
480.0     15
300.0     13
84.0       4
240.0      4
120.0      3
36.0       2
60.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [342]:
ltrain_df['Credit_History'].value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [343]:
#Replaced Nulls with 0
ltrain_df['Credit_History'].fillna(value = 0, inplace = True)
ltrain_df['Credit_History'].value_counts()

1.0    475
0.0    139
Name: Credit_History, dtype: int64

In [344]:
ltrain_df.isnull().sum()

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

In [345]:
#Drop remaining Nulls, less than 5%
ltrain_df.dropna(inplace = True)
ltrain_df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

Begin Test dataset cleaning

In [346]:
ltest_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 367 entries, LP001015 to LP002989
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             356 non-null    object 
 1   Married            367 non-null    object 
 2   Dependents         357 non-null    object 
 3   Education          367 non-null    object 
 4   Self_Employed      344 non-null    object 
 5   ApplicantIncome    367 non-null    int64  
 6   CoapplicantIncome  367 non-null    int64  
 7   LoanAmount         362 non-null    float64
 8   Loan_Amount_Term   361 non-null    float64
 9   Credit_History     338 non-null    float64
 10  Property_Area      367 non-null    object 
dtypes: float64(3), int64(2), object(6)
memory usage: 34.4+ KB


In [347]:
ltest_df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,367.000000,367.000000,362.000000,361.000000,338.000000
mean,4805.599455,1569.577657,136.132597,342.537396,0.825444
std,4910.685399,2334.232099,61.366652,65.156643,0.380150
min,0.000000,0.000000,28.000000,6.000000,0.000000
25%,2864.000000,0.000000,100.250000,360.000000,1.000000
50%,3786.000000,1025.000000,125.000000,360.000000,1.000000
75%,5060.000000,2430.500000,158.000000,360.000000,1.000000
max,72529.000000,24000.000000,550.000000,480.000000,1.000000


In [348]:
ltest_df.duplicated().any()

True

In [349]:
ltest_df.duplicated().sum()

1

In [350]:
ltest_df.drop_duplicates(inplace = True)
ltest_df.duplicated().any()

False

In [351]:
ltest_df.shape

(366, 11)

In [352]:
ltest_df['Gender'].value_counts()

Male      285
Female     70
Name: Gender, dtype: int64

In [353]:
#Replaced Nulls with majority value
ltest_df['Gender'].fillna(value = 'Male', inplace = True)
ltest_df['Gender'].value_counts()

Male      296
Female     70
Name: Gender, dtype: int64

In [354]:
ltest_df['Married'].value_counts()

Yes    233
No     133
Name: Married, dtype: int64

In [355]:
ltest_df['Dependents'].value_counts()

0     199
2      59
1      58
3+     40
Name: Dependents, dtype: int64

In [356]:
#Replaced Nulls with 0
ltest_df['Dependents'].fillna(value = 0, inplace = True)
ltest_df['Dependents'].value_counts()

0     199
2      59
1      58
3+     40
0      10
Name: Dependents, dtype: int64

In [357]:
ltest_df['Education'].value_counts()

Graduate        282
Not Graduate     84
Name: Education, dtype: int64

In [358]:
ltest_df['Self_Employed'].value_counts()

No     307
Yes     36
Name: Self_Employed, dtype: int64

In [359]:
#Replaced Nulls with No
ltest_df['Self_Employed'].fillna(value = 'No', inplace = True)
ltest_df['Self_Employed'].value_counts()

No     330
Yes     36
Name: Self_Employed, dtype: int64

In [360]:
ltest_df['Property_Area'].value_counts()

Urban        139
Semiurban    116
Rural        111
Name: Property_Area, dtype: int64

In [361]:
ltest_df['Loan_Amount_Term'].value_counts()

360.0    310
180.0     22
480.0      8
300.0      7
240.0      4
84.0       3
6.0        1
120.0      1
36.0       1
350.0      1
12.0       1
60.0       1
Name: Loan_Amount_Term, dtype: int64

In [362]:
#Replaced Nulls with majority value
ltest_df['Loan_Amount_Term'].fillna(value = 360, inplace = True)
ltest_df['Loan_Amount_Term'].value_counts()

360.0    316
180.0     22
480.0      8
300.0      7
240.0      4
84.0       3
6.0        1
120.0      1
36.0       1
350.0      1
12.0       1
60.0       1
Name: Loan_Amount_Term, dtype: int64

In [363]:
ltest_df['Credit_History'].value_counts()

1.0    278
0.0     59
Name: Credit_History, dtype: int64

In [364]:
#Replaced Nulls with 0
ltest_df['Credit_History'].fillna(value = 0, inplace = True)
ltest_df['Credit_History'].value_counts()

1.0    278
0.0     88
Name: Credit_History, dtype: int64

In [365]:
ltest_df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           5
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [366]:
#Drop remaining Nulls, less than 5%
ltest_df.dropna(inplace = True)
ltest_df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

Step 2:

Examine and Visualize Data